# <font color='brown'>楼 + 机器学习实战</font>

# 挑战：训练手写字符识别神经网络

---

** <center><font color='red'>以下内容仅保留挑战代码部分，挑战全文请到原课程查看。</font></center>**

---

**<font color='red'>挑战</font>：使用 `1x5` 的子图样式绘制 Digits 数据集前 `5` 个手写字符的图像。**

In [ ]:
### 代码开始 ### (3～5 行代码)
fig, axes = plt.subplots(1, 5, figsize=(12,4))
for i, image in enumerate(digits.images[:5]):
    axes[i].imshow(image, cmap=plt.cm.gray_r)
### 代码结束 ###

---

**<font color='red'>挑战</font>：使用 `train_test_split()` 将数据集切分为 80%（训练集） 和 20%（测试集） 两部分。**

**<font color='blue'>规定</font>**：训练集特征，训练集目标，测试集特征，测试集目标分别定义为：`train_x, train_y, test_x, test_y`，随机数种子定为 `30`。

In [ ]:
### 代码开始 ### (≈ 2 行代码)
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(digits.data, digits.target, test_size=0.2, random_state=30)
### 代码结束 ###

---

**<font color='red'>挑战</font>：使用 `MLPClassifier()` 搭建神经网络结构，并训练手写字符识别模型，最后得到在测试集上的预测准确率。**

**<font color='blue'>规定</font>**：

- 神经网络结构包含 `2` 个隐含层，依次有 `100` 和 `50` 个神经元。
- 使用 `relu` 作为激活函数。
- 使用随机梯度下降 `SGD` 方法求解。
- 学习率为 `0.02` 且在学习过程中保持不变。
- 最大迭代次数为 `100` 次。
- 随机数种子设为 `1`。
- 其余参数使用默认值。

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

def mpl():
    """
    参数:无

    返回:
    model -- 人工神经网络模型
    score -- 测试集上的预测准确率
    """
    ### 代码开始 ### (≈ 2 行代码)
    model = MLPClassifier(
        hidden_layer_sizes=(100, 50),
        activation='relu',
        solver='sgd',
        learning_rate_init=0.02,
        learning_rate='constant',
        max_iter=100,
        random_state=1
    )

    model.fit(train_x, train_y)
    score = accuracy_score(test_y, model.predict(test_x))
    ### 代码结束 ###
    return model, score

---

<div style="color: #999;font-size: 12px;font-style: italic;">*本课程内容，由作者授权实验楼发布，未经允许，禁止转载、下载及非法传播。</div>